Для первой задачи мы используем данные [Jester Online Joke Recommender System](https://goldberg.berkeley.edu/jester-data/)

**Описание данных**

Файл `train_joke_df.csv` содержит:
- UID - id пользователей
- JID - id шуток, которые 
- Ratin - рейтинг шутки, который проставил пользователь 


Рейтинг имеет значение от -10.00 до 10.00. Могут встречаться значения 99.00, но это обозначает Null (нет рейтинга от пользователя).

Метрика для оценки [RMSE](https://www.codecamp.ru/blog/how-to-interpret-rmse/)

Минимальный RMSE: `4.2238`



In [1]:
#!pip install "scikit-surprise==1.1.3"

In [2]:
#!pip install "xlrd==2.0.1"

### Import

In [3]:
import numpy as np
import pandas as pd
from collections import defaultdict
from surprise import Dataset, Reader, KNNWithMeans, accuracy
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from sklearn.model_selection import train_test_split as tts
from surprise.model_selection import KFold

np.random.seed(42)

### Базовые функции для скоринга и получения рекомендаций

In [4]:
def get_num_user_ratings(uid):
    """ возвращает кол-во рейтингов у пользователя 
    args: 
      uid: id пользователей
    returns: 
      кол-во объектов, которые оценил пользователь
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # пользователя не было во время обучения (новый, отправить на стартовые рекомендации)
        return 0
    
def get_num_item_ratings(iid):
    """ возвращает кол-во пользователей, которые оценили выбранный элемент 
    args:
      iid: строка с элементов рекомендации
    returns:
      кол-во пользователей, которые дали оценки по элементу
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
# На основе Surprise FAQ построим рекомендации Топ-N
def get_top_n(predictions, n=5):
    """Определят Топ-N рекомендаций

    Args:
        predictions(list of Prediction objects): Списко рекомендаций, из алгоритма Surprise
        n(int): Кол-во топ рекомендаций

    Returns:
        Словарь пользователь - список рекомендакиций для пользователей
        [(raw item id, rating estimation), ...]
    """

    # Предикт для каждого пользователя
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Сортировка предикта (по пользователям)
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n    

### Загрузка и обработка данных

In [5]:
df = pd.read_csv(r'data\recsys-in-practice\train_joke_df.csv')

df.head(5)

,UID,JID,Rating
0,18029,6,-1.26
1,3298,64,-4.17
2,3366,58,0.92
3,12735,92,3.69
4,11365,38,-6.60


In [6]:
# сделаем сортировку и перепишем index
df = df.sort_values(by=['UID', 'JID'])
df = df.reset_index(drop=True)

In [7]:
# создадим на основе набора данных
# поднабор, который требуется для библиотеки Surprise

# указываем минимальный и максимальный рейтинги
reader = Reader(rating_scale=(-10, 10))

# передаём набор, указывая последовательность колонок: user (raw) ids, item (raw) ids, ratings
# для Surprise - это обязательно
data = Dataset.load_from_df(df[['UID', 'JID', 'Rating']], reader)

In [8]:
trainset_data = data.build_full_trainset()

# сделаем разделение на обучающую и тестовую выборку
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

### Обучение модели

In [9]:
# определим набор данных для GridSearchCV
sim_options = {
    "name": ["msd", "cosine"], # способы оценки похожести (в GridSearch)
    "min_support": [3, 4],     # минимальное кол-во общих пользоватлей с данной шуткой
    "user_based": [False],     # поиск "похожести" будет на основе шуток, а не пользователей
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=2,)
gs.fit(data)
     
# результат
print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
4.279031273142975
{'sim_options': {'name': 'cosine', 'min_support': 3, 'user_based': False}}


In [10]:
# обучим с лучшими параметрами
algo = gs.best_estimator['rmse']
algo.fit(trainset)

# получим предикт и посмотрим метрику
predictions = algo.test(testset)
accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 4.2173


4.217254761536634

### Тестирование и результаты

In [11]:
# получаем предикт
uid = 1  # id пользователя 
iid = 1  # iв шутки

# получим предик на основе обученных данных
# -7.82 - это фактический рейтинг, но посмотрим, какой ответ будет в предикте
pred = algo.predict(uid, iid, r_ui=-7.82, verbose=True)

user: 1          item: 1          r_ui = -7.82   est = -3.08   {'actual_k': 40, 'was_impossible': False}


In [12]:
uid = 24983  # id пользователя 
iid = 62     # iв шутки

pred = algo.predict(uid, iid, r_ui=-0.29, verbose=True)

user: 24983      item: 62         r_ui = -0.29   est = 5.19   {'actual_k': 40, 'was_impossible': False}


### Обзор рекомендаций

In [13]:
# построим таблицу для обзора набора рекомендаций
# посмотрим, какие элементы и в каком кол-ве рекомендуем
trainset = algo.trainset

predictions_df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])

predictions_df['№ кол-во пользовательских рейтингов'] = predictions_df.uid.apply(get_num_user_ratings)
predictions_df['№ кол-во рейтингов элементов'] = predictions_df.iid.apply(get_num_item_ratings)
predictions_df['error'] = abs(predictions_df.est - predictions_df.rui)

best_predictions = predictions_df.sort_values(by='error')[:10]
worst_predictions = predictions_df.sort_values(by='error')[-10:]

In [14]:
best_predictions.head(5)

,uid,iid,rui,est,details,№ кол-во пользовательских рейтингов,№ кол-во рейтингов элементов,error
117673,6770,53,0.44,0.439985,"{'actual_k': 40, 'was_impossible': False}",51,15931,0.000015
84263,15986,62,4.71,4.709969,"{'actual_k': 25, 'was_impossible': False}",29,16014,0.000031
75416,17382,48,0.63,0.630055,"{'actual_k': 40, 'was_impossible': False}",62,15747,0.000055
94118,20763,85,3.93,3.930074,"{'actual_k': 40, 'was_impossible': False}",65,6016,0.000074
78694,3700,45,4.42,4.419881,"{'actual_k': 24, 'was_impossible': False}",24,13758,0.000119


In [15]:
# Предикт для всех, кого нет в выборке для обучения
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions)

# Сделаем вывод рекомендаций
a=0
for uid, user_ratings in top_n.items():
    a+=1
    print(uid, [iid for (iid, _) in user_ratings])
    
    if a==10:
        break

19208 [89, 29, 62, 54, 35]
8671 [32, 27, 72, 49, 68]
6037 [89, 36, 27, 72, 49]
3233 [50, 36, 27, 31, 76]
3449 [50, 89, 53, 72, 49]
10032 [32, 35, 29, 53, 72]
5774 [89, 32, 29, 72, 68]
23392 [89, 36, 62, 29, 35]
3039 [89, 36, 62, 27, 53]
17395 [89, 50, 27, 72, 35]


### Для отправки на тестирование

In [33]:
test = pd.read_csv(r'data\recsys-in-practice\test_joke_df_nofactrating.csv', index_col=0)
test.head(5)

,UID,JID
InteractionID,,
0,11228,39
1,21724,85
2,16782,56
3,12105,42
4,14427,2


In [34]:
test['Rating'] = test[['UID', 'JID']].apply(lambda x: algo.predict(x[0], x[1], verbose=False).est,
                                                      axis = 1)
                                                      


In [35]:
# вид набора данных, который должен быть отправлен для тестирования
test['Rating'].to_frame().head(5)

,Rating
InteractionID,
0,0.826231
1,-5.278602
2,-1.332138
3,7.300694
4,6.709370


In [36]:
# формирование файла для отправки в Kaggle
test['Rating'].to_frame().to_csv('baseline.csv')

In [37]:
df_user_group = df.groupby('UID').agg({'Rating':'mean'}).rename(columns={'Rating':'Rating_mean'})
df_user_group

,Rating_mean
UID,
1,-3.466774
2,2.544881
3,7.047073
4,2.440256
5,3.098500
...,...
24979,1.944048
24980,1.986184
24981,-1.875385


In [38]:
df_joke_group = df.groupby('JID').agg({'Rating':'mean'}).rename(columns={'Rating':'Rating_mean'})
df_joke_group

,Rating_mean
JID,
1,0.937282
2,0.190557
3,0.340709
4,-1.448563
5,0.419578
...,...
96,1.495376
97,1.700951
98,0.752163


In [39]:
df_user_group[np.abs(df_user_group['Rating_mean'])> 7]

,Rating_mean
UID,
3,7.047073
183,-7.052075
210,7.065366
468,7.329355
616,7.519103
...,...
24498,7.281266
24556,7.670154
24620,-7.565510


In [40]:
test = test.join(df_user_group, on='UID')
test

,UID,JID,Rating,Rating_mean
InteractionID,,,,
0,11228,39,0.826231,0.406129
1,21724,85,-5.278602,-3.935714
2,16782,56,-1.332138,-2.209747
3,12105,42,7.300694,6.441163
4,14427,2,6.709370,6.293051
...,...,...,...,...
362086,3085,66,1.089143,0.487368
362087,13765,31,2.856345,1.747273
362088,10341,29,5.265267,2.569434


In [41]:
test[test['UID'] == 5]

,UID,JID,Rating,Rating_mean
InteractionID,,,,
1275,5,5,3.571999,3.0985
30805,5,34,4.327722,3.0985
40168,5,6,4.959394,3.0985
109159,5,17,0.696426,3.0985
119674,5,10,4.401963,3.0985
136806,5,97,4.997907,3.0985
138623,5,60,1.547504,3.0985
140037,5,91,5.059354,3.0985
197240,5,29,6.161103,3.0985


In [42]:
test = test.rename(columns={'Rating':'Rating_old'})

In [43]:
test['Rating'] = (test['Rating_old'] + test['Rating_mean']) / 2

In [44]:
test

,UID,JID,Rating_old,Rating_mean,Rating
InteractionID,,,,,
0,11228,39,0.826231,0.406129,0.616180
1,21724,85,-5.278602,-3.935714,-4.607158
2,16782,56,-1.332138,-2.209747,-1.770942
3,12105,42,7.300694,6.441163,6.870929
4,14427,2,6.709370,6.293051,6.501210
...,...,...,...,...,...
362086,3085,66,1.089143,0.487368,0.788256
362087,13765,31,2.856345,1.747273,2.301809
362088,10341,29,5.265267,2.569434,3.917351


In [45]:
# вид набора данных, который должен быть отправлен для тестирования
test['Rating'].to_frame().head(5)

,Rating
InteractionID,
0,0.616180
1,-4.607158
2,-1.770942
3,6.870929
4,6.501210


In [46]:
# формирование файла для отправки в Kaggle
test['Rating'].to_frame().to_csv('baseline_mean.csv')

In [47]:
algo.predict(11228, 39, verbose=False)

Prediction(uid=11228, iid=39, r_ui=None, est=0.8262309301836691, details={'actual_k': 40, 'was_impossible': False})

In [48]:
algo.estimate(u=11228, i=39)

(0.7425443267984426, {'actual_k': 24})

In [49]:
algo.sim.shape

(100, 100)